In [1]:
import pandas as pd
import json
import pickle
import os
import itertools
from ast import literal_eval
import numpy as np
import sys
from datetime import datetime
from IPython.display import clear_output, display
import subprocess
from datetime import datetime
pd.options.display.float_format = '{:.0f}'.format

In [2]:
def LoadJsonFile(filename): 
    with open(filename, 'r') as f:
        DicConfig = json.load(f)
    return DicConfig


def GlobalDicDeplier(OneDic):
    for k,v in OneDic.items():
        exec('globals()[k] = v')
    return None

In [3]:
DicConfig = LoadJsonFile(os.path.join(os.getcwd(),"config.json"))
GlobalDicDeplier(DicConfig)
sys.path.append(Root)
from fun import *

print("Load Config variables")

Load Config variables


In [4]:
path = os.path.join(Root,FolderProject,"RefFam.pkl")
RefFam = LoadPickleOrInit(path)

path = os.path.join(Root,FolderProject,"RefRT.pkl")
RefRT = LoadPickleOrInit(path)

In [8]:
path = os.path.join(Root,FolderProject,"ISdf.pkl")
ISdf = LoadPickleOrInit(path)

In [10]:
ISdf = ISdf[ISdf.Rank<3]

In [11]:
ISdf

,BassineID,AUTHORTWEETID,f,Rank,cumsumf,sum,p,BassineSize
0,438903,1221436953385672704,173,1,173,624,28,3600
1,438903,1221446628617355264,43,2,216,624,35,3600
32,438904,1221454832701231104,75,1,75,746,10,3600
33,438904,1221451508606218240,62,2,137,746,18,3600
71,438905,1221468048143585280,93,1,93,599,16,3600
72,438905,1221470984018046720,62,2,155,599,26,3600


In [5]:
RefFam.shape

(342, 6)

In [6]:
RefRT.shape

(3442, 6)

In [12]:
RefRT.status.value_counts()

ok    2270
ko    1172
Name: status, dtype: int64